# Total mediation effects

Based on the paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4287269/


In [1]:
%cd /mnt/home/zzhang/ceph/jemm

/mnt/ceph/users/zzhang/CHARM-SexDiff-Splicing/jemm


In [2]:
import pandas as pd
import statsmodels.api as sm

In [3]:
data = pd.read_table("data-V9/mediation_analysis/first_mid.forCMA.txt")

In [4]:
data.head()

,sid,pid,final,Age,Sex,is_female,base_ISG_LV3,base_ISG_LV5,base_ISG_LV6,base_ISG_LV10,...,msk,nv,rn_nose,sob,subj_fvr,taste,throat,num_symps,CTavg,days_since_infect
0,20_0149-T42,149,First,19.0,M,0,-0.563513,0.144323,-0.089108,-0.083181,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.957500,0
1,20_0152-T53,152,First,19.0,M,0,-0.756253,0.070571,-0.205008,-0.073074,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.025000,0
2,20_0155-T42,155,First,22.0,M,0,-1.059343,0.325916,-0.360406,-0.130774,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.643333,0
3,20_0156-T42,156,First,22.0,M,0,-0.408753,0.458178,-0.379783,-0.444856,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.715000,0
4,20_0157-T42,157,First,19.0,M,0,0.002407,0.249973,-0.218508,-0.407287,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,5.0,25.736667,0


In [5]:
data.columns

Index(['sid', 'pid', 'final', 'Age', 'Sex', 'is_female', 'base_ISG_LV3',
       'base_ISG_LV5', 'base_ISG_LV6', 'base_ISG_LV10', 'base_ISG_LV3_prj',
       'base_ISG_LV5_prj', 'base_ISG_LV6_prj', 'base_ISG_LV10_prj',
       'ISG_LV3_prj', 'ISG_LV5_prj', 'ISG_LV6_prj', 'ISG_LV10_prj',
       'PSI_CD45RBC', 'delta_PSI_CD45RBC', 'asymp', 'abd', 'chill', 'cough',
       'diarr', 'fatigue', 'fvr', 'headache', 'msk', 'nv', 'rn_nose', 'sob',
       'subj_fvr', 'taste', 'throat', 'num_symps', 'CTavg',
       'days_since_infect'],
      dtype='object')

In [6]:
# E[Y|a,m,c]
Y = 'CTavg' 
a = 'is_female'
Ms = ['base_ISG_LV3','base_ISG_LV5','base_ISG_LV6','base_ISG_LV10']
Cs = ['days_since_infect']
df = data.query('final=="First"')[[Y, a, ] + Ms + Cs]
df['intercept'] = 1
df.dropna(inplace=True)
m1 = sm.OLS(df[Y], df[['intercept', a]+ Ms + Cs])
res1 = m1.fit()
thetas = res1.summary2().tables[1].loc[Ms, 'Coef.'].to_dict()
res1.summary2().tables[1]

/mnt/home/zzhang/anaconda3/envs/rna-seq/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1830: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/mnt/home/zzhang/anaconda3/envs/rna-seq/lib/python3.6/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
intercept,25.082487,0.462360,54.248843,2.436307e-124,24.170949,25.994025
is_female,2.118668,1.450610,1.460536,1.456588e-01,-0.741195,4.978531
base_ISG_LV3,0.081927,1.123434,0.072926,9.419355e-01,-2.132912,2.296767
base_ISG_LV5,0.090326,2.030042,0.044494,9.645532e-01,-3.911882,4.092533
base_ISG_LV6,2.213871,1.806240,1.225679,2.217122e-01,-1.347115,5.774857
base_ISG_LV10,0.080075,2.136255,0.037484,9.701353e-01,-4.131532,4.291682
days_since_infect,0.000000,0.000000,NaN,NaN,0.000000,0.000000


In [7]:
m2 = sm.OLS(df[Y], df[['intercept', a] + Cs])
res2 = m2.fit()
res2.summary2().tables[1]

/mnt/home/zzhang/anaconda3/envs/rna-seq/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1830: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/mnt/home/zzhang/anaconda3/envs/rna-seq/lib/python3.6/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
intercept,24.899706,0.429953,57.912655,1.673099e-131,24.052153,25.747259
is_female,2.633627,1.308418,2.012833,4.540304e-02,0.054380,5.212874
days_since_infect,0.000000,0.000000,NaN,NaN,0.000000,0.000000


In [8]:
# E[m|a,c]
betas = {}
for M in Ms:
    m = sm.OLS(df[M], df[['intercept', a] + Cs])
    res = m.fit()
    betas[M] = res.summary2().tables[1].loc[a, 'Coef.']

/mnt/home/zzhang/anaconda3/envs/rna-seq/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1830: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/mnt/home/zzhang/anaconda3/envs/rna-seq/lib/python3.6/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


In [9]:
# indirect
sum([thetas[M]*betas[M] for M in Ms])

0.5149589206236083

In [10]:
# direct
res1.summary2().tables[1].loc[a, 'Coef.']

2.1186682723588617